### Kim Tam

In [10]:
source('RegDI.R')

In [11]:
# ---- Cargar librerías necesarias ----
library(survey)
library(dplyr)

# ---- Definir la función para calcular theta_PDI ----
calcular_theta_pdi <- function(datos, var_A, var_B) {
  N <- nrow(datos)
  Nb <- sum(datos$muestra_B == 1)
  
  # Sumatoria para muestra B
  sum_B <- sum(datos[[var_B]][datos$muestra_B == 1], na.rm = TRUE)
  
  # Media para la parte de la población no cubierta por muestra B
  mean_A <- mean(datos[[var_A]][datos$muestra_A == 1 & datos$muestra_B == 0], na.rm = TRUE)
  
  # Calcular theta_PDI
  theta_PDI <- (1 / N) * (sum_B + (N - Nb) * mean_A)
  
  return(theta_PDI)
}

# ---- Función para generar los datos ----
generar_datos <- function(n) {
  xi <- rnorm(n, mean = 2, sd = 1) # Generamos xi a partir de una distribución normal N(2, 1)
  ei <- rnorm(n, mean = 0, sd = sqrt(0.51)) # Generamos ei a partir de una distribución normal N(0, 0.51)
  yi <- 3 + 0.7 * (xi - 2) + ei # Calculamos yi
  ui <- rnorm(n, mean = 0, sd = 0.5) # Generamos ui a partir de una normal N(0, 0.5^2)
  yi_ast <- 2 + 0.9 * (yi - 3) + ui # Calculamos yi_ast (con error de medición)
  datos <- data.frame(xi = xi, yi = yi, yi_ast = yi_ast)
  return(datos)
}

# ---- Función para seleccionar muestras A y B ----
seleccionar_muestras <- function(datos, size_a, size_muestra_B) {
  n <- nrow(datos)
  
  # Muestra A (probabilística)
  datos$muestra_A <- 0
  indices_seleccionados <- sample(1:n, size = size_a)
  datos$muestra_A[indices_seleccionados] <- 1
  
  # Muestra B (no probabilística)
  grupo_1 <- which(datos$xi <= 2)  # Indices del grupo con xi <= 2
  grupo_2 <- which(datos$xi > 2)   # Indices del grupo con xi > 2
  n_grupo_1 <- floor((3/5) * size_muestra_B)  # 3/5 para grupo_1
  n_grupo_2 <- size_muestra_B - n_grupo_1     # 2/5 para grupo_2
  
  # Asegurar que hay suficientes unidades en cada grupo
  if(length(grupo_1) < n_grupo_1 | length(grupo_2) < n_grupo_2){
    stop("No hay suficientes unidades en alguno de los grupos para seleccionar la muestra B.")
  }
  
  muestras_grupo_1 <- sample(grupo_1, size = n_grupo_1)
  muestras_grupo_2 <- sample(grupo_2, size = n_grupo_2)
  datos$muestra_B <- 0
  datos$muestra_B[muestras_grupo_1] <- 1
  datos$muestra_B[muestras_grupo_2] <- 1
  
  return(datos)
}

# ---- Simulación de Monte Carlo ----
simulacion_montecarlo <- function(n_reps, size_a, size_muestra_B, n) {
  resultados <- data.frame(
    mean_A_esc1 = numeric(n_reps),
    mean_B_esc1 = numeric(n_reps),
    mean_A_esc2 = numeric(n_reps),
    mean_B_esc2 = numeric(n_reps),
    mean_A_esc3 = numeric(n_reps),
    mean_B_esc3 = numeric(n_reps),
    theta_PDI_esc1 = numeric(n_reps),
    theta_PDI_esc2 = numeric(n_reps),
    theta_PDI_esc3 = numeric(n_reps),
    RegDI_esc1 = numeric(n_reps),
    RegDI_var_esc1 = numeric(n_reps),         # Nueva columna
    RegDI_esc2 = numeric(n_reps),
    RegDI_var_esc2 = numeric(n_reps),         # Nueva columna
    RegDI_Correccion_esc2 = numeric(n_reps),
    RegDI_Corr_var_esc2 = numeric(n_reps),    # Nueva columna
    RegDI_esc3 = numeric(n_reps),
    RegDI_var_esc3 = numeric(n_reps),         # Nueva columna
    RegDI_Correccion_esc3 = numeric(n_reps),
    RegDI_Corr_var_esc3 = numeric(n_reps),    # Nueva columna
    diff_A_esc1 = numeric(n_reps),
    diff_B_esc1 = numeric(n_reps),
    diff_A_esc2 = numeric(n_reps),
    diff_B_esc2 = numeric(n_reps),
    diff_A_esc3 = numeric(n_reps),
    diff_B_esc3 = numeric(n_reps),
    diff_PDI_esc1 = numeric(n_reps),
    diff_PDI_esc2 = numeric(n_reps),
    diff_PDI_esc3 = numeric(n_reps),
    diff_RegDI_esc1 = numeric(n_reps),
    diff_RegDI_esc2 = numeric(n_reps),
    diff_RegDI_Correccion_esc2 = numeric(n_reps),
    diff_RegDI_esc3 = numeric(n_reps),
    diff_RegDI_Correccion_esc3 = numeric(n_reps),
    SE_RegDI_esc1 = numeric(n_reps),         # Nueva columna para SE
    SE_RegDI_esc2 = numeric(n_reps),         # Nueva columna para SE
    SE_RegDI_Correccion_esc2 = numeric(n_reps), # Nueva columna para SE
    SE_RegDI_esc3 = numeric(n_reps),         # Nueva columna para SE
    SE_RegDI_Correccion_esc3 = numeric(n_reps)  # Nueva columna para SE
  )
  
  for (rep in 1:n_reps) {
    # Generamos los datos y seleccionamos muestras
    datos_simul <- generar_datos(n)
    datos_simul <- seleccionar_muestras(datos_simul, size_a, size_muestra_B)
    
    # Cálculo de las medias en muestra A y B en los tres escenarios
    # Escenario 1: Sin errores de medición
    mean_A_esc1 <- mean(datos_simul$yi[datos_simul$muestra_A == 1], na.rm = TRUE)
    mean_B_esc1 <- mean(datos_simul$yi[datos_simul$muestra_B == 1], na.rm = TRUE)
    
    # Escenario 2: Errores en la muestra B
    mean_A_esc2 <- mean(datos_simul$yi[datos_simul$muestra_A == 1], na.rm = TRUE)
    mean_B_esc2 <- mean(datos_simul$yi_ast[datos_simul$muestra_B == 1], na.rm = TRUE)
    
    # Escenario 3: Errores en la muestra A
    mean_A_esc3 <- mean(datos_simul$yi_ast[datos_simul$muestra_A == 1], na.rm = TRUE)
    mean_B_esc3 <- mean(datos_simul$yi[datos_simul$muestra_B == 1], na.rm = TRUE)
    
    # Cálculo de theta_PDI en los tres escenarios
    theta_PDI_esc1 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi")
    theta_PDI_esc2 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi_ast")
    theta_PDI_esc3 <- calcular_theta_pdi(datos_simul, var_A = "yi_ast", var_B = "yi")
    
    # Cálculo de RegDI en los tres escenarios
    # Escenario 1: Sin errores de medición
    regdi_esc1 <- RegDI(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc1 <- regdi_esc1$mean_RegDI
    Var_RegDI_esc1 <- regdi_esc1$var_RegDI          # Obtener la varianza
    SE_RegDI_esc1 <- sqrt(Var_RegDI_esc1)           # Calcular SE
    
    # Escenario 2: Errores en la muestra B
    # Sin corrección
    regdi_esc2 <- RegDI(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi_ast", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc2 <- regdi_esc2$mean_RegDI
    Var_RegDI_esc2 <- regdi_esc2$var_RegDI
    SE_RegDI_esc2 <- sqrt(Var_RegDI_esc2)
    
    # Con corrección
    regdi_correccion_esc2 <- RegDI(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi_ast", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 1
    )
    T_RegDI_Corr_esc2 <- regdi_correccion_esc2$mean_RegDI
    Var_RegDI_Corr_esc2 <- regdi_correccion_esc2$var_RegDI
    SE_RegDI_Correccion_esc2 <- sqrt(Var_RegDI_Corr_esc2)
    
    # Escenario 3: Errores en la muestra A
    # Sin corrección
    regdi_esc3 <- RegDI(
      data = datos_simul, 
      y_A_col = "yi_ast", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc3 <- regdi_esc3$mean_RegDI
    Var_RegDI_esc3 <- regdi_esc3$var_RegDI
    SE_RegDI_esc3 <- sqrt(Var_RegDI_esc3)
    
    # Con corrección
    regdi_correccion_esc3 <- RegDI(
      data = datos_simul, 
      y_A_col = "yi_ast", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 2
    )
    T_RegDI_Corr_esc3 <- regdi_correccion_esc3$mean_RegDI
    Var_RegDI_Corr_esc3 <- regdi_correccion_esc3$var_RegDI
    SE_RegDI_Correccion_esc3 <- sqrt(Var_RegDI_Corr_esc3)
    
    # Guardamos los resultados de las medias y theta_PDI
    resultados$mean_A_esc1[rep] <- mean_A_esc1
    resultados$mean_B_esc1[rep] <- mean_B_esc1
    resultados$mean_A_esc2[rep] <- mean_A_esc2
    resultados$mean_B_esc2[rep] <- mean_B_esc2
    resultados$mean_A_esc3[rep] <- mean_A_esc3
    resultados$mean_B_esc3[rep] <- mean_B_esc3
    resultados$theta_PDI_esc1[rep] <- theta_PDI_esc1
    resultados$theta_PDI_esc2[rep] <- theta_PDI_esc2
    resultados$theta_PDI_esc3[rep] <- theta_PDI_esc3
    
    # Guardamos los resultados de RegDI
    resultados$RegDI_esc1[rep] <- T_RegDI_esc1
    resultados$RegDI_var_esc1[rep] <- Var_RegDI_esc1
    resultados$RegDI_esc2[rep] <- T_RegDI_esc2
    resultados$RegDI_var_esc2[rep] <- Var_RegDI_esc2
    resultados$RegDI_Correccion_esc2[rep] <- T_RegDI_Corr_esc2
    resultados$RegDI_Corr_var_esc2[rep] <- Var_RegDI_Corr_esc2
    resultados$RegDI_esc3[rep] <- T_RegDI_esc3
    resultados$RegDI_var_esc3[rep] <- Var_RegDI_esc3
    resultados$RegDI_Correccion_esc3[rep] <- T_RegDI_Corr_esc3
    resultados$RegDI_Corr_var_esc3[rep] <- Var_RegDI_Corr_esc3
    
    # Guardamos los SE de RegDI
    resultados$SE_RegDI_esc1[rep] <- SE_RegDI_esc1
    resultados$SE_RegDI_esc2[rep] <- SE_RegDI_esc2
    resultados$SE_RegDI_Correccion_esc2[rep] <- SE_RegDI_Correccion_esc2
    resultados$SE_RegDI_esc3[rep] <- SE_RegDI_esc3
    resultados$SE_RegDI_Correccion_esc3[rep] <- SE_RegDI_Correccion_esc3
    
    # Calculamos la media poblacional verdadera
    media_poblacion <- mean(datos_simul$yi)
    
    # Cálculo de las diferencias (sesgo) con respecto a la media poblacional
    resultados$diff_A_esc1[rep] <- mean_A_esc1 - media_poblacion
    resultados$diff_B_esc1[rep] <- mean_B_esc1 - media_poblacion
    resultados$diff_A_esc2[rep] <- mean_A_esc2 - media_poblacion
    resultados$diff_B_esc2[rep] <- mean_B_esc2 - media_poblacion
    resultados$diff_A_esc3[rep] <- mean_A_esc3 - media_poblacion
    resultados$diff_B_esc3[rep] <- mean_B_esc3 - media_poblacion
    resultados$diff_PDI_esc1[rep] <- theta_PDI_esc1 - media_poblacion
    resultados$diff_PDI_esc2[rep] <- theta_PDI_esc2 - media_poblacion
    resultados$diff_PDI_esc3[rep] <- theta_PDI_esc3 - media_poblacion
    resultados$diff_RegDI_esc1[rep] <- T_RegDI_esc1 - media_poblacion
    resultados$diff_RegDI_esc2[rep] <- T_RegDI_esc2 - media_poblacion
    resultados$diff_RegDI_Correccion_esc2[rep] <- T_RegDI_Corr_esc2 - media_poblacion
    resultados$diff_RegDI_esc3[rep] <- T_RegDI_esc3 - media_poblacion
    resultados$diff_RegDI_Correccion_esc3[rep] <- T_RegDI_Corr_esc3 - media_poblacion
  }
  
  return(resultados)
}


# ---- Definir los parámetros de simulación ----
n <- 10000            # Tamaño de la población
size_a <- 1000        # Tamaño de la muestra A
size_muestra_B <- 5000  # Tamaño de la muestra B
n_reps <- 1000        # Número de repeticiones de la simulación

# ---- Ejecutar la simulación de Monte Carlo ----
set.seed(123)  # Para reproducibilidad
resultados_simulacion <- simulacion_montecarlo(n_reps, size_a, size_muestra_B, n)
resultados_simulacion

mean_A_esc1,mean_B_esc1,mean_A_esc2,mean_B_esc2,mean_A_esc3,mean_B_esc3,theta_PDI_esc1,theta_PDI_esc2,theta_PDI_esc3,RegDI_esc1,⋯,diff_RegDI_esc1,diff_RegDI_esc2,diff_RegDI_Correccion_esc2,diff_RegDI_esc3,diff_RegDI_Correccion_esc3,SE_RegDI_esc1,SE_RegDI_esc2,SE_RegDI_Correccion_esc2,SE_RegDI_esc3,SE_RegDI_Correccion_esc3
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.972227,2.874595,2.972227,1.875342,1.976628,2.874595,2.990362,2.490735,2.489971,2.990362,⋯,-0.0014746253,-0.0080281993,-0.0080281993,-0.9989390,0.007846988,1.0186303,1.0186405,1.0186405,1.0498566,1.167849
3.103020,2.887583,3.103020,1.900211,2.089132,2.887583,3.052767,2.559081,2.549980,3.052767,⋯,0.0469084362,0.0689066483,0.0689066483,-0.9630662,0.069418809,1.0248397,1.0239892,1.0239892,1.0588493,1.171928
3.024339,2.878594,3.024339,1.883679,2.021793,2.878594,3.019079,2.521621,2.511765,3.019079,⋯,0.0240461627,0.0241781759,0.0241781759,-0.9781162,0.022916991,1.0121680,1.0124371,1.0124371,1.0653854,1.159722
2.992218,2.887132,2.992218,1.897170,1.999341,2.887132,3.014539,2.519558,2.513155,3.014539,⋯,0.0004411873,-0.0058546190,-0.0058546190,-0.9945969,0.005602425,0.9978892,0.9990851,0.9990851,1.0363124,1.147954
2.992364,2.884663,2.992364,1.900002,2.000156,2.884663,2.986571,2.494240,2.476681,2.986571,⋯,-0.0003100047,-0.0102112512,-0.0102112512,-0.9906842,-0.024600938,0.9944966,0.9962583,0.9962583,1.0460399,1.119812
3.006682,2.888268,3.006682,1.909637,1.992850,2.888268,3.041557,2.552241,2.524911,3.041557,⋯,0.0347271529,0.0391644050,0.0391644050,-0.9820542,0.029119165,1.0512752,1.0505946,1.0505946,1.0653301,1.150249
3.024716,2.879456,3.024716,1.896410,2.016484,2.879456,2.996175,2.504651,2.488699,2.996175,⋯,-0.0014001993,0.0106037893,0.0106037893,-1.0064117,0.002213758,1.0076964,1.0079163,1.0079163,1.0399462,1.171951
2.947249,2.868050,2.947249,1.875566,1.920265,2.868050,2.936648,2.440406,2.415923,2.936648,⋯,-0.0468242165,-0.0356581894,-0.0356581894,-1.0732105,-0.056029188,0.9998026,0.9992512,0.9992512,1.0363529,1.149791
3.041836,2.885801,3.041836,1.901156,2.020177,2.885801,3.040212,2.547890,2.522652,3.040212,⋯,0.0340331833,0.0410756416,0.0410756416,-0.9869676,0.024852145,1.0394123,1.0385788,1.0385788,1.0909759,1.129388


In [12]:
# ---- Presentación de resultados en tablas ----

# Promedios de las diferencias con respecto a la población
promedios <- resultados_simulacion %>%
  summarise(
    Mean_A_Esc1 = mean(diff_A_esc1),
    Mean_B_Esc1 = mean(diff_B_esc1),
    Theta_PDI_Esc1 = mean(diff_PDI_esc1),
    RegDI_Esc1 = mean(diff_RegDI_esc1),
    Mean_A_Esc2 = mean(diff_A_esc2),
    Mean_B_Esc2 = mean(diff_B_esc2),
    Theta_PDI_Esc2 = mean(diff_PDI_esc2),
    RegDI_Esc2 = mean(diff_RegDI_esc2),
    RegDI_Correccion_Esc2 = mean(diff_RegDI_Correccion_esc2),
    Mean_A_Esc3 = mean(diff_A_esc3),
    Mean_B_Esc3 = mean(diff_B_esc3),
    Theta_PDI_Esc3 = mean(diff_PDI_esc3),
    RegDI_Esc3 = mean(diff_RegDI_esc3),
    RegDI_Correccion_Esc3 = mean(diff_RegDI_Correccion_esc3)
  )

# Desviación estándar de las diferencias
desviaciones <- resultados_simulacion %>%
  summarise(
    SD_A_Esc1 = sd(diff_A_esc1),
    SD_B_Esc1 = sd(diff_B_esc1),
    SD_Theta_PDI_Esc1 = sd(diff_PDI_esc1),
    SD_RegDI_Esc1 = sd(diff_RegDI_esc1),
    SD_A_Esc2 = sd(diff_A_esc2),
    SD_B_Esc2 = sd(diff_B_esc2),
    SD_Theta_PDI_Esc2 = sd(diff_PDI_esc2),
    SD_RegDI_Esc2 = sd(diff_RegDI_esc2),
    SD_RegDI_Correccion_Esc2 = sd(diff_RegDI_Correccion_esc2),
    SD_A_Esc3 = sd(diff_A_esc3),
    SD_B_Esc3 = sd(diff_B_esc3),
    SD_Theta_PDI_Esc3 = sd(diff_PDI_esc3),
    SD_RegDI_Esc3 = sd(diff_RegDI_esc3),
    SD_RegDI_Correccion_Esc3 = sd(diff_RegDI_Correccion_esc3)
  )

# ---- Organización de los resultados en un solo data.frame ----

# Crear un data frame con los resultados
resultados_finales <- data.frame(
  Escenario = character(),
  Estimador = character(),
  Sesgo = numeric(),
  Desviacion = numeric(),
  stringsAsFactors = FALSE
)

# Escenario 1
resultados_finales <- resultados_finales %>%
  add_row(
    Escenario = "Esc 1",
    Estimador = "A",
    Sesgo = promedios$Mean_A_Esc1,
    Desviacion = desviaciones$SD_A_Esc1
  ) %>%
  add_row(
    Escenario = "Esc 1",
    Estimador = "B",
    Sesgo = promedios$Mean_B_Esc1,
    Desviacion = desviaciones$SD_B_Esc1
  ) %>%
  add_row(
    Escenario = "Esc 1",
    Estimador = "PDI",
    Sesgo = promedios$Theta_PDI_Esc1,
    Desviacion = desviaciones$SD_Theta_PDI_Esc1
  ) %>%
  add_row(
    Escenario = "Esc 1",
    Estimador = "RegDI",
    Sesgo = promedios$RegDI_Esc1,
    Desviacion = desviaciones$SD_RegDI_Esc1
  )

# Escenario 2
resultados_finales <- resultados_finales %>%
  add_row(
    Escenario = "Esc 2",
    Estimador = "A",
    Sesgo = promedios$Mean_A_Esc2,
    Desviacion = desviaciones$SD_A_Esc2
  ) %>%
  add_row(
    Escenario = "Esc 2",
    Estimador = "B",
    Sesgo = promedios$Mean_B_Esc2,
    Desviacion = desviaciones$SD_B_Esc2
  ) %>%
  add_row(
    Escenario = "Esc 2",
    Estimador = "PDI",
    Sesgo = promedios$Theta_PDI_Esc2,
    Desviacion = desviaciones$SD_Theta_PDI_Esc2
  ) %>%
  add_row(
    Escenario = "Esc 2",
    Estimador = "RegDI",
    Sesgo = promedios$RegDI_Esc2,
    Desviacion = desviaciones$SD_RegDI_Esc2
  ) %>%
  add_row(
    Escenario = "Esc 2",
    Estimador = "RegDI_Correccion",
    Sesgo = promedios$RegDI_Correccion_Esc2,
    Desviacion = desviaciones$SD_RegDI_Correccion_Esc2
  )

# Escenario 3
resultados_finales <- resultados_finales %>%
  add_row(
    Escenario = "Esc 3",
    Estimador = "A",
    Sesgo = promedios$Mean_A_Esc3,
    Desviacion = desviaciones$SD_A_Esc3
  ) %>%
  add_row(
    Escenario = "Esc 3",
    Estimador = "B",
    Sesgo = promedios$Mean_B_Esc3,
    Desviacion = desviaciones$SD_B_Esc3
  ) %>%
  add_row(
    Escenario = "Esc 3",
    Estimador = "PDI",
    Sesgo = promedios$Theta_PDI_Esc3,
    Desviacion = desviaciones$SD_Theta_PDI_Esc3
  ) %>%
  add_row(
    Escenario = "Esc 3",
    Estimador = "RegDI",
    Sesgo = promedios$RegDI_Esc3,
    Desviacion = desviaciones$SD_RegDI_Esc3
  ) %>%
  add_row(
    Escenario = "Esc 3",
    Estimador = "RegDI_Correccion",
    Sesgo = promedios$RegDI_Correccion_Esc3,
    Desviacion = desviaciones$SD_RegDI_Correccion_Esc3
  )

# ---- Redondear los valores a 2 decimales ----
resultados_finales <- resultados_finales %>%
  mutate(
    Sesgo = round(Sesgo, 2),
    Desviacion = round(Desviacion, 2)
  )

# ---- Mostrar el data.frame final ----
print(resultados_finales)


   Escenario        Estimador Sesgo Desviacion
1      Esc 1                A  0.00       0.03
2      Esc 1                B -0.11       0.01
3      Esc 1              PDI  0.00       0.02
4      Esc 1            RegDI  0.00       0.02
5      Esc 2                A  0.00       0.03
6      Esc 2                B -1.10       0.01
7      Esc 2              PDI -0.49       0.02
8      Esc 2            RegDI  0.00       0.02
9      Esc 2 RegDI_Correccion  0.00       0.02
10     Esc 3                A -1.00       0.03
11     Esc 3                B -0.11       0.01
12     Esc 3              PDI -0.50       0.02
13     Esc 3            RegDI -1.00       0.03
14     Esc 3 RegDI_Correccion  0.00       0.03


------------------


### Simulación PC

In [10]:
source('PC.R')

In [11]:
# Cargar librerías necesarias
library(survey)
library(dplyr)
library(knitr)
library(kableExtra)

# Definir las funciones PC_Estimator_Scenario1 y RegDI según lo proporcionado
# [Asegúrate de haber ejecutado las definiciones de PC_Estimator_Scenario1 y RegDI antes de este script]

# Parámetros de la simulación
n_sim <- 100       # Número de simulaciones
N <- 10000             # Tamaño de la población
size_a <- 1000         # Tamaño de la muestra probabilística A
size_muestra_B <- 5000 # Tamaño de la muestra no probabilística B

# Inicializar matrices para almacenar los resultados
estimates_mean_S_A <- numeric(n_sim)
estimates_mean_S_B <- numeric(n_sim)
estimates_RegDI <- numeric(n_sim)
estimates_RegDI_X1 <- numeric(n_sim)
estimates_RegDI_e1 <- numeric(n_sim)
estimates_PC_X1 <- numeric(n_sim)
estimates_PC_e1 <- numeric(n_sim)

# Valor verdadero de la media poblacional
Y_true <- 3

# Iniciar la simulación
for (sim in 1:n_sim) {
  # Generar la población
  x_i <- rnorm(N, mean = 2, sd = 1)
  eta_i <- rnorm(N, mean = 0, sd = sqrt(0.51))
  y_i <- 3 + 0.7 * (x_i - 2) + eta_i
  
  # Generar la versión contaminada de y_i (no se usará en el Escenario I)
  tilde_y_i <- 2 + 0.9 * (y_i - 3) + rnorm(N, mean = 0, sd = 0.5)
  
  # Generar variables auxiliares e_i y las variables indicadoras e1_i y e2_i
  rho <- 0.5
  v_i <- rnorm(N, mean = 0, sd = 1)
  e_i <- rho * x_i + sqrt(1 - rho^2) * v_i  # Variable correlacionada
  e1_i <- ifelse(e_i <= 1, 1, 0) # Variable indicadora e1i
  e2_i <- ifelse(e_i > 1, 1, 0) # Variable indicadora e2i
  
  # Generar las variables auxiliares x1_i y x2_i
  x1_i <- ifelse(x_i <= 2, 1, 0) 
  x2_i <- ifelse(x_i > 2, 1, 0)
  
  # Crear data.frame de la población
  poblacion <- data.frame(
    id = 1:N,
    x_i = x_i,
    y_i = y_i,
    tilde_y_i = tilde_y_i,
    e_i = e_i,
    e1_i = e1_i,
    e2_i = e2_i,
    x1_i = x1_i,
    x2_i = x2_i
  )
  
  # Seleccionar muestra probabilística A (S_A) de tamaño 1000
  set.seed(sim + 1000)  # Asegurar reproducibilidad
  indices_A <- sample(1:N, size = size_a, replace = FALSE)
  poblacion$muestra_A <- 0
  poblacion$muestra_A[indices_A] <- 1
  
  # Seleccionar muestra no probabilística B (S_B) de tamaño 5000
  # Estratificación basada en x_i
  stratum1 <- which(poblacion$x_i <= 2)
  stratum2 <- which(poblacion$x_i > 2)
  n_B1 <- 3000
  n_B2 <- 2000
  set.seed(sim + 2000)  # Asegurar reproducibilidad
  indices_B1 <- sample(stratum1, size = n_B1, replace = FALSE)
  indices_B2 <- sample(stratum2, size = n_B2, replace = FALSE)
  indices_B <- c(indices_B1, indices_B2)
  poblacion$muestra_B <- 0
  poblacion$muestra_B[indices_B] <- 1
  
  # Calcular los totales poblacionales conocidos para las variables auxiliares
  pop_totals_x <- c(
    x1_i = sum(poblacion$x1_i),
    x2_i = sum(poblacion$x2_i)
  )
  
  pop_totals_e <- c(
    e1_i = sum(poblacion$e1_i),
    e2_i = sum(poblacion$e2_i)
  )
  
  # Calcular Mean_S_A y Mean_S_B
  mean_S_A <- mean(poblacion$y_i[poblacion$muestra_A == 1])
  mean_S_B <- mean(poblacion$y_i[poblacion$muestra_B == 1])
  
  estimates_mean_S_A[sim] <- mean_S_A
  estimates_mean_S_B[sim] <- mean_S_B
  
  # Calcular RegDI sin variables auxiliares
  result_RegDI <- RegDI(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = NULL,
    weights_A_col = NULL
  )
  estimates_RegDI[sim] <- result_RegDI$mean_RegDI
  
  # Calcular RegDI con (x1_i) para evitar colinealidad
  result_RegDI_X1 <- RegDI(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("x1_i"),  # Solo una variable auxiliar
    weights_A_col = NULL
  )
  estimates_RegDI_X1[sim] <- result_RegDI_X1$mean_RegDI
  
  # Calcular RegDI con (e1_i) para evitar colinealidad
  result_RegDI_e1 <- RegDI(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("e1_i"),  # Solo una variable auxiliar
    weights_A_col = NULL
  )
  estimates_RegDI_e1[sim] <- result_RegDI_e1$mean_RegDI
  
  # Calcular PC con (x1_i)
  PC_X1 <- PC_Estimator(
    data = poblacion,
    y_B_col = "y_i",
    sample_B_col = "muestra_B",
    aux_vars = c("x1_i","x2_i"),
    pop_totals = pop_totals_x,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL
  )
  estimates_PC_X1[sim] <- PC_X1$mean_PC
  
  # Calcular PC con (e1_i)
  PC_e1 <- PC_Estimator(
    data = poblacion,
    y_B_col = "y_i",
    sample_B_col = "muestra_B",
    aux_vars = c("e1_i","e2_i"),
    pop_totals = pop_totals_e,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL
  )
  estimates_PC_e1[sim] <- PC_e1$mean_PC
  
  # Opcional: imprimir progreso
  if (sim %% 100 == 0) {
    cat("Simulación", sim, "completada.\n")
  }
}

# Calcular las estadísticas resumen para cada estimador
calculate_stats <- function(estimates, true_value) {
  bias <- mean(estimates) - true_value
  se <- sd(estimates)
  rmse <- sqrt(mean((estimates - true_value)^2))
  return(c(bias = bias, se = se, rmse = rmse))
}

# Crear una lista con todos los estimadores
estimators <- list(
  Mean_S_A = estimates_mean_S_A,
  Mean_S_B = estimates_mean_S_B,
  RegDI = estimates_RegDI,
  RegDI_X1 = estimates_RegDI_X1,
  RegDI_e1 = estimates_RegDI_e1,
  PC_X1 = estimates_PC_X1,
  PC_e1 = estimates_PC_e1
)

# Calcular Bias, SE y RMSE para cada estimador
results_summary <- data.frame(
  Estimator = character(),
  Bias = numeric(),
  SE = numeric(),
  RMSE = numeric(),
  stringsAsFactors = FALSE
)

for (est_name in names(estimators)) {
  stats <- calculate_stats(estimators[[est_name]], Y_true)
  results_summary <- rbind(results_summary, data.frame(
    Estimator = est_name,
    Bias = round(stats["bias"], 4),
    SE = round(stats["se"], 4),
    RMSE = round(stats["rmse"], 4),
    stringsAsFactors = FALSE
  ))
}

# Agregar la columna de Escenario (I)
results_summary <- results_summary %>%
  mutate(Scenario = "I") %>%
  select(Scenario, Estimator, Bias, SE, RMSE)

# Ordenar la tabla por Estimador
results_summary <- results_summary %>%
  arrange(Estimator)

# Mostrar los resultados en una tabla
print(results_summary)

ERROR: Error in RegDI(data = poblacion, y_A_col = "y_i", y_B_col = "y_i", size_a = size_a, : no se pudo encontrar la función "RegDI"


--------------------------------

RegDI segun chatGPT

In [12]:
source('RegDI2.R')

In [13]:
# ---- Cargar librerías necesarias ----
library(survey)
library(dplyr)

# ---- Definir la función para calcular theta_PDI ----
calcular_theta_pdi <- function(datos, var_A, var_B) {
  N <- nrow(datos)
  Nb <- sum(datos$muestra_B == 1)
  
  # Sumatoria para muestra B
  sum_B <- sum(datos[[var_B]][datos$muestra_B == 1], na.rm = TRUE)
  
  # Media para la parte de la población no cubierta por muestra B
  mean_A <- mean(datos[[var_A]][datos$muestra_A == 1 & datos$muestra_B == 0], na.rm = TRUE)
  
  # Calcular theta_PDI
  theta_PDI <- (1 / N) * (sum_B + (N - Nb) * mean_A)
  
  return(theta_PDI)
}

# ---- Función para generar los datos ----
generar_datos <- function(n) {
  xi <- rnorm(n, mean = 2, sd = 1) # Generamos xi a partir de una distribución normal N(2, 1)
  ei <- rnorm(n, mean = 0, sd = sqrt(0.51)) # Generamos ei a partir de una distribución normal N(0, 0.51)
  yi <- 3 + 0.7 * (xi - 2) + ei # Calculamos yi
  ui <- rnorm(n, mean = 0, sd = 0.5) # Generamos ui a partir de una normal N(0, 0.5^2)
  yi_ast <- 2 + 0.9 * (yi - 3) + ui # Calculamos yi_ast (con error de medición)
  datos <- data.frame(xi = xi, yi = yi, yi_ast = yi_ast)
  return(datos)
}

# ---- Función para seleccionar muestras A y B ----
seleccionar_muestras <- function(datos, size_a, size_muestra_B) {
  n <- nrow(datos)
  
  # Muestra A (probabilística)
  datos$muestra_A <- 0
  indices_seleccionados <- sample(1:n, size = size_a)
  datos$muestra_A[indices_seleccionados] <- 1
  
  # Muestra B (no probabilística)
  grupo_1 <- which(datos$xi <= 2)  # Indices del grupo con xi <= 2
  grupo_2 <- which(datos$xi > 2)   # Indices del grupo con xi > 2
  n_grupo_1 <- floor((3/5) * size_muestra_B)  # 3/5 para grupo_1
  n_grupo_2 <- size_muestra_B - n_grupo_1     # 2/5 para grupo_2
  
  # Asegurar que hay suficientes unidades en cada grupo
  if(length(grupo_1) < n_grupo_1 | length(grupo_2) < n_grupo_2){
    stop("No hay suficientes unidades en alguno de los grupos para seleccionar la muestra B.")
  }
  
  muestras_grupo_1 <- sample(grupo_1, size = n_grupo_1)
  muestras_grupo_2 <- sample(grupo_2, size = n_grupo_2)
  datos$muestra_B <- 0
  datos$muestra_B[muestras_grupo_1] <- 1
  datos$muestra_B[muestras_grupo_2] <- 1
  
  return(datos)
}

# ---- Ejecutar una sola iteración de la simulación ----
run_single_iteration <- function(size_a, size_muestra_B, n) {
  # Generamos los datos y seleccionamos muestras
  datos_simul <- generar_datos(n)
  datos_simul <- seleccionar_muestras(datos_simul, size_a, size_muestra_B)

  # Diseño de encuesta para la muestra A y muestra B para cada escenario
  diseño_A_esc1 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_A == 1, ], weights = ~1)
  diseño_B_esc1 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_B == 1, ], weights = ~1)

  diseño_A_esc2 <- diseño_A_esc1  # Muestra A no cambia para el escenario 2
  diseño_B_esc2 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_B == 1, ], weights = ~1)
  
  diseño_A_esc3 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_A == 1, ], weights = ~1)
  diseño_B_esc3 <- diseño_B_esc1  # Muestra B no cambia para el escenario 3
    
  # Cálculo de las medias en muestra A y B en los tres escenarios
  # Escenario 1: Sin errores de medición
  media_A_esc1 <- svymean(~yi, diseño_A_esc1)
  media_B_esc1 <- svymean(~yi, diseño_B_esc1)
  
  # Escenario 2: Errores en la muestra B (yi_ast en lugar de yi en muestra B)
  media_A_esc2 <- svymean(~yi, diseño_A_esc2)  # Muestra A sin cambios
  media_B_esc2 <- svymean(~yi_ast, diseño_B_esc2)  # Errores en la muestra B
  
  # Escenario 3: Errores en la muestra A
  media_A_esc3 <- svymean(~yi_ast, diseño_A_esc3)  # Errores en la muestra A
  media_B_esc3 <- svymean(~yi, diseño_B_esc3)  # Muestra B sin cambios

  # Calcular theta_PDI para verificar
  theta_PDI_esc1 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi")
  theta_PDI_esc2 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi_ast")
  theta_PDI_esc3 <- calcular_theta_pdi(datos_simul, var_A = "yi_ast", var_B = "yi")
  
  # Mostrar resultados de la iteración
  print("Escenario 1")
  print(media_A_esc1)
  print(media_B_esc1)
  print(paste("Theta PDI Escenario 1:", theta_PDI_esc1))
  
  print("Escenario 2")
  print(media_A_esc2)
  print(media_B_esc2)
  print(paste("Theta PDI Escenario 2:", theta_PDI_esc2))
  
  print("Escenario 3")
  print(media_A_esc3)
  print(media_B_esc3)
  print(paste("Theta PDI Escenario 3:", theta_PDI_esc3))
}

# ---- Definir los parámetros de la simulación ----
n <- 10000          # Tamaño de la población
size_a <- 1000        # Tamaño de la muestra A
size_muestra_B <- 5000  # Tamaño de la muestra B

# Ejecutar una sola iteración
run_single_iteration(size_a, size_muestra_B, n)


[1] "Escenario 1"
     mean    SE
yi 2.9931 0.031
     mean     SE
yi 2.8716 0.0141
[1] "Theta PDI Escenario 1: 3.00186148573023"
[1] "Escenario 2"
     mean    SE
yi 2.9931 0.031
         mean     SE
yi_ast 1.8841 0.0144
[1] "Theta PDI Escenario 2: 2.5081403816312"
[1] "Escenario 3"
         mean     SE
yi_ast 2.0036 0.0323
     mean     SE
yi 2.8716 0.0141
[1] "Theta PDI Escenario 3: 2.49041551334404"


In [14]:
# ---- Cargar librerías necesarias ----
library(survey)
library(dplyr)

# ---- Definir la función para calcular theta_PDI ----
calcular_theta_pdi <- function(datos, var_A, var_B) {
  N <- nrow(datos)
  Nb <- sum(datos$muestra_B == 1)
  
  # Sumatoria para muestra B
  sum_B <- sum(datos[[var_B]][datos$muestra_B == 1], na.rm = TRUE)
  
  # Media para la parte de la población no cubierta por muestra B
  mean_A <- mean(datos[[var_A]][datos$muestra_A == 1 & datos$muestra_B == 0], na.rm = TRUE)
  
  # Calcular theta_PDI
  theta_PDI <- (1 / N) * (sum_B + (N - Nb) * mean_A)
  
  return(theta_PDI)
}

# ---- Función para generar los datos ----
generar_datos <- function(n) {
  xi <- rnorm(n, mean = 2, sd = 1) # Generamos xi a partir de una distribución normal N(2, 1)
  ei <- rnorm(n, mean = 0, sd = sqrt(0.51)) # Generamos ei a partir de una distribución normal N(0, 0.51)
  yi <- 3 + 0.7 * (xi - 2) + ei # Calculamos yi
  ui <- rnorm(n, mean = 0, sd = 0.5) # Generamos ui a partir de una normal N(0, 0.5^2)
  yi_ast <- 2 + 0.9 * (yi - 3) + ui # Calculamos yi_ast (con error de medición)
  datos <- data.frame(xi = xi, yi = yi, yi_ast = yi_ast)
  return(datos)
}

# ---- Función para seleccionar muestras A y B ----
seleccionar_muestras <- function(datos, size_a, size_muestra_B) {
  n <- nrow(datos)
  
  # Muestra A (probabilística)
  datos$muestra_A <- 0
  indices_seleccionados <- sample(1:n, size = size_a)
  datos$muestra_A[indices_seleccionados] <- 1
  
  # Muestra B (no probabilística)
  grupo_1 <- which(datos$xi <= 2)  # Indices del grupo con xi <= 2
  grupo_2 <- which(datos$xi > 2)   # Indices del grupo con xi > 2
  n_grupo_1 <- floor((3/5) * size_muestra_B)  # 3/5 para grupo_1
  n_grupo_2 <- size_muestra_B - n_grupo_1     # 2/5 para grupo_2
  
  # Asegurar que hay suficientes unidades en cada grupo
  if(length(grupo_1) < n_grupo_1 | length(grupo_2) < n_grupo_2){
    stop("No hay suficientes unidades en alguno de los grupos para seleccionar la muestra B.")
  }
  
  muestras_grupo_1 <- sample(grupo_1, size = n_grupo_1)
  muestras_grupo_2 <- sample(grupo_2, size = n_grupo_2)
  datos$muestra_B <- 0
  datos$muestra_B[muestras_grupo_1] <- 1
  datos$muestra_B[muestras_grupo_2] <- 1
  
  return(datos)
}

# ---- Simulación de Monte Carlo ----
simulacion_montecarlo <- function(n_reps, size_a, size_muestra_B, n) {
  resultados <- data.frame(
    mean_A_esc1 = numeric(n_reps),
    mean_B_esc1 = numeric(n_reps),
    mean_A_esc2 = numeric(n_reps),
    mean_B_esc2 = numeric(n_reps),
    mean_A_esc3 = numeric(n_reps),
    mean_B_esc3 = numeric(n_reps),
    SE_A_esc1 = numeric(n_reps),
    SE_B_esc1 = numeric(n_reps),
    SE_A_esc2 = numeric(n_reps),
    SE_B_esc2 = numeric(n_reps),
    SE_A_esc3 = numeric(n_reps),
    SE_B_esc3 = numeric(n_reps),
    theta_PDI_esc1 = numeric(n_reps),
    theta_PDI_esc2 = numeric(n_reps),
    theta_PDI_esc3 = numeric(n_reps),
    RegDI_esc1 = numeric(n_reps),
    RegDI_var_esc1 = numeric(n_reps),         # Varianza del estimador
    RegDI_esc2 = numeric(n_reps),
    RegDI_var_esc2 = numeric(n_reps),         # Varianza del estimador
    RegDI_Correccion_esc2 = numeric(n_reps),
    RegDI_Corr_var_esc2 = numeric(n_reps),    # Varianza del estimador
    RegDI_esc3 = numeric(n_reps),
    RegDI_var_esc3 = numeric(n_reps),         # Varianza del estimador
    RegDI_Correccion_esc3 = numeric(n_reps),
    RegDI_Corr_var_esc3 = numeric(n_reps),    # Varianza del estimador
    diff_A_esc1 = numeric(n_reps),
    diff_B_esc1 = numeric(n_reps),
    diff_A_esc2 = numeric(n_reps),
    diff_B_esc2 = numeric(n_reps),
    diff_A_esc3 = numeric(n_reps),
    diff_B_esc3 = numeric(n_reps),
    diff_PDI_esc1 = numeric(n_reps),
    diff_PDI_esc2 = numeric(n_reps),
    diff_PDI_esc3 = numeric(n_reps),
    diff_RegDI_esc1 = numeric(n_reps),
    diff_RegDI_esc2 = numeric(n_reps),
    diff_RegDI_Correccion_esc2 = numeric(n_reps),
    diff_RegDI_esc3 = numeric(n_reps),
    diff_RegDI_Correccion_esc3 = numeric(n_reps)
  )
  
  for (rep in 1:n_reps) {
    # Generamos los datos y seleccionamos muestras
    datos_simul <- generar_datos(n)
    datos_simul <- seleccionar_muestras(datos_simul, size_a, size_muestra_B)

    # Definir los FPC para las muestras A y B
    fpc_A <- rep(n, size_a)
    fpc_B <- rep(n, size_muestra_B)

    # Diseño de encuesta para la muestra A y muestra B en los tres escenarios con FPC
    diseño_A_esc1 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_A == 1, ], weights = ~1, fpc = fpc_A)
    diseño_B_esc1 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_B == 1, ], weights = ~1, fpc = fpc_B)

    diseño_A_esc2 <- diseño_A_esc1  # Muestra A no cambia para el escenario 2
    diseño_B_esc2 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_B == 1, ], weights = ~1, fpc = fpc_B)
    
    diseño_A_esc3 <- svydesign(ids = ~1, data = datos_simul[datos_simul$muestra_A == 1, ], weights = ~1, fpc = fpc_A)
    diseño_B_esc3 <- diseño_B_esc1  # Muestra B no cambia para el escenario 3
      
    # Cálculo de las medias en muestra A y B en los tres escenarios
    # Escenario 1: Sin errores de medición
    media_A_esc1 <- svymean(~yi, diseño_A_esc1)
    media_B_esc1 <- svymean(~yi, diseño_B_esc1)
    
    # Escenario 2: Errores en la muestra B (yi_ast en lugar de yi en muestra B)
    media_A_esc2 <- svymean(~yi, diseño_A_esc2)  # Muestra A sin cambios
    media_B_esc2 <- svymean(~yi_ast, diseño_B_esc2)  # Errores en la muestra B
    
    # Escenario 3: Errores en la muestra A
    media_A_esc3 <- svymean(~yi_ast, diseño_A_esc3)  # Errores en la muestra A
    media_B_esc3 <- svymean(~yi, diseño_B_esc3)  # Muestra B sin cambios
    
    # Guardar medias y errores estándar
    resultados$mean_A_esc1[rep] <- coef(media_A_esc1)
    resultados$SE_A_esc1[rep] <- SE(media_A_esc1)
    resultados$mean_B_esc1[rep] <- coef(media_B_esc1)
    resultados$SE_B_esc1[rep] <- SE(media_B_esc1)

    resultados$mean_A_esc2[rep] <- coef(media_A_esc2)
    resultados$SE_A_esc2[rep] <- SE(media_A_esc2)
    resultados$mean_B_esc2[rep] <- coef(media_B_esc2)
    resultados$SE_B_esc2[rep] <- SE(media_B_esc2)

    resultados$mean_A_esc3[rep] <- coef(media_A_esc3)
    resultados$SE_A_esc3[rep] <- SE(media_A_esc3)
    resultados$mean_B_esc3[rep] <- coef(media_B_esc3)
    resultados$SE_B_esc3[rep] <- SE(media_B_esc3)

    # Cálculo de theta_PDI en los tres escenarios
    theta_PDI_esc1 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi")
    theta_PDI_esc2 <- calcular_theta_pdi(datos_simul, var_A = "yi", var_B = "yi_ast")
    theta_PDI_esc3 <- calcular_theta_pdi(datos_simul, var_A = "yi_ast", var_B = "yi")
    
    # Cálculo de RegDI en los tres escenarios
    # Escenario 1: Sin errores de medición
    regdi_esc1 <- RegDI2(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc1 <- regdi_esc1$mean_RegDI
    Var_RegDI_esc1 <- regdi_esc1$var_RegDI          # Obtener la varianza
    
    # Escenario 2: Errores en la muestra B
    # Sin corrección
    regdi_esc2 <- RegDI2(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi_ast", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc2 <- regdi_esc2$mean_RegDI
    Var_RegDI_esc2 <- regdi_esc2$var_RegDI
    
    # Con corrección
    regdi_correccion_esc2 <- RegDI2(
      data = datos_simul, 
      y_A_col = "yi", 
      y_B_col = "yi_ast", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 1
    )
    T_RegDI_Corr_esc2 <- regdi_correccion_esc2$mean_RegDI
    Var_RegDI_Corr_esc2 <- regdi_correccion_esc2$var_RegDI
    
    # Escenario 3: Errores en la muestra A
    # Sin corrección
    regdi_esc3 <- RegDI2(
      data = datos_simul, 
      y_A_col = "yi_ast", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 0
    )
    T_RegDI_esc3 <- regdi_esc3$mean_RegDI
    Var_RegDI_esc3 <- regdi_esc3$var_RegDI
    
    # Con corrección
    regdi_correccion_esc3 <- RegDI2(
      data = datos_simul, 
      y_A_col = "yi_ast", 
      y_B_col = "yi", 
      size_a = size_a, 
      size_muestra_B = size_muestra_B, 
      N = n,
      apply_correction = 2
    )
    T_RegDI_Corr_esc3 <- regdi_correccion_esc3$mean_RegDI
    Var_RegDI_Corr_esc3 <- regdi_correccion_esc3$var_RegDI
    
    # Guardamos los resultados de theta_PDI
    resultados$theta_PDI_esc1[rep] <- theta_PDI_esc1
    resultados$theta_PDI_esc2[rep] <- theta_PDI_esc2
    resultados$theta_PDI_esc3[rep] <- theta_PDI_esc3
    
    # Guardamos los resultados de RegDI
    resultados$RegDI_esc1[rep] <- T_RegDI_esc1
    resultados$RegDI_var_esc1[rep] <- Var_RegDI_esc1
    resultados$RegDI_esc2[rep] <- T_RegDI_esc2
    resultados$RegDI_var_esc2[rep] <- Var_RegDI_esc2
    resultados$RegDI_Correccion_esc2[rep] <- T_RegDI_Corr_esc2
    resultados$RegDI_Corr_var_esc2[rep] <- Var_RegDI_Corr_esc2
    resultados$RegDI_esc3[rep] <- T_RegDI_esc3
    resultados$RegDI_var_esc3[rep] <- Var_RegDI_esc3
    resultados$RegDI_Correccion_esc3[rep] <- T_RegDI_Corr_esc3
    resultados$RegDI_Corr_var_esc3[rep] <- Var_RegDI_Corr_esc3
    
    # Calculamos la media poblacional verdadera
    media_poblacion <- mean(datos_simul$yi)
    
    # Cálculo de las diferencias (sesgo) con respecto a la media poblacional
    resultados$diff_A_esc1[rep] <- coef(media_A_esc1) - media_poblacion
    resultados$diff_B_esc1[rep] <- coef(media_B_esc1) - media_poblacion
    resultados$diff_A_esc2[rep] <- coef(media_A_esc2) - media_poblacion
    resultados$diff_B_esc2[rep] <- coef(media_B_esc2) - media_poblacion
    resultados$diff_A_esc3[rep] <- coef(media_A_esc3) - media_poblacion
    resultados$diff_B_esc3[rep] <- coef(media_B_esc3) - media_poblacion
    resultados$diff_PDI_esc1[rep] <- theta_PDI_esc1 - media_poblacion
    resultados$diff_PDI_esc2[rep] <- theta_PDI_esc2 - media_poblacion
    resultados$diff_PDI_esc3[rep] <- theta_PDI_esc3 - media_poblacion
    resultados$diff_RegDI_esc1[rep] <- T_RegDI_esc1 - media_poblacion
    resultados$diff_RegDI_esc2[rep] <- T_RegDI_esc2 - media_poblacion
    resultados$diff_RegDI_Correccion_esc2[rep] <- T_RegDI_Corr_esc2 - media_poblacion
    resultados$diff_RegDI_esc3[rep] <- T_RegDI_esc3 - media_poblacion
    resultados$diff_RegDI_Correccion_esc3[rep] <- T_RegDI_Corr_esc3 - media_poblacion
  }
  
  return(resultados)
}

# ---- Definir los parámetros de simulación ----
n <- 10000          # Tamaño de la población ajustado al del artículo
size_a <- 1000        # Tamaño de la muestra A
size_muestra_B <- 5000  # Tamaño de la muestra B
n_reps <- 1000        # Número de repeticiones de la simulación

# ---- Ejecutar la simulación de Monte Carlo ----
set.seed(123)  # Para reproducibilidad
resultados_simulacion <- simulacion_montecarlo(n_reps, size_a, size_muestra_B, n)

# ---- Presentación de resultados en tablas ----

# Promedios de las diferencias con respecto a la población (sesgo)
promedios <- resultados_simulacion %>%
  summarise(
    Mean_A_Esc1 = mean(diff_A_esc1),
    Mean_B_Esc1 = mean(diff_B_esc1),
    Theta_PDI_Esc1 = mean(diff_PDI_esc1),
    RegDI_Esc1 = mean(diff_RegDI_esc1),
    Mean_A_Esc2 = mean(diff_A_esc2),
    Mean_B_Esc2 = mean(diff_B_esc2),
    Theta_PDI_Esc2 = mean(diff_PDI_esc2),
    RegDI_Esc2 = mean(diff_RegDI_esc2),
    RegDI_Correccion_Esc2 = mean(diff_RegDI_Correccion_esc2),
    Mean_A_Esc3 = mean(diff_A_esc3),
    Mean_B_Esc3 = mean(diff_B_esc3),
    Theta_PDI_Esc3 = mean(diff_PDI_esc3),
    RegDI_Esc3 = mean(diff_RegDI_esc3),
    RegDI_Correccion_Esc3 = mean(diff_RegDI_Correccion_esc3)
  )

# Varianza promedio de los estimadores
varianzas <- resultados_simulacion %>%
  summarise(
    Var_RegDI_Esc1 = mean(RegDI_var_esc1),
    Var_RegDI_Esc2 = mean(RegDI_var_esc2),
    Var_RegDI_Correccion_Esc2 = mean(RegDI_Corr_var_esc2),
    Var_RegDI_Esc3 = mean(RegDI_var_esc3),
    Var_RegDI_Correccion_Esc3 = mean(RegDI_Corr_var_esc3)
  )

# Calcular el SE promedio de los estimadores
SE_promedios <- resultados_simulacion %>%
  summarise(
    SE_RegDI_Esc1 = mean(sqrt(RegDI_var_esc1)),
    SE_RegDI_Esc2 = mean(sqrt(RegDI_var_esc2)),
    SE_RegDI_Correccion_Esc2 = mean(sqrt(RegDI_Corr_var_esc2)),
    SE_RegDI_Esc3 = mean(sqrt(RegDI_var_esc3)),
    SE_RegDI_Correccion_Esc3 = mean(sqrt(RegDI_Corr_var_esc3))
  )

# Calcular el RMSE de los estimadores
RMSEs <- data.frame(
  RMSE_RegDI_Esc1 = sqrt((promedios$RegDI_Esc1)^2 + varianzas$Var_RegDI_Esc1),
  RMSE_RegDI_Esc2 = sqrt((promedios$RegDI_Esc2)^2 + varianzas$Var_RegDI_Esc2),
  RMSE_RegDI_Correccion_Esc2 = sqrt((promedios$RegDI_Correccion_Esc2)^2 + varianzas$Var_RegDI_Correccion_Esc2),
  RMSE_RegDI_Esc3 = sqrt((promedios$RegDI_Esc3)^2 + varianzas$Var_RegDI_Esc3),
  RMSE_RegDI_Correccion_Esc3 = sqrt((promedios$RegDI_Correccion_Esc3)^2 + varianzas$Var_RegDI_Correccion_Esc3)
)

# Organizar los resultados en un data.frame
resultados_finales <- data.frame(
  Escenario = c("Esc 1", "Esc 1", "Esc 1", "Esc 1",
                "Esc 2", "Esc 2", "Esc 2", "Esc 2", "Esc 2",
                "Esc 3", "Esc 3", "Esc 3", "Esc 3", "Esc 3"),
  Estimador = c("A", "B", "PDI", "RegDI",
                "A", "B", "PDI", "RegDI", "RegDI_Correccion",
                "A", "B", "PDI", "RegDI", "RegDI_Correccion"),
  Sesgo = c(
    promedios$Mean_A_Esc1, promedios$Mean_B_Esc1, promedios$Theta_PDI_Esc1, promedios$RegDI_Esc1,
    promedios$Mean_A_Esc2, promedios$Mean_B_Esc2, promedios$Theta_PDI_Esc2, promedios$RegDI_Esc2, promedios$RegDI_Correccion_Esc2,
    promedios$Mean_A_Esc3, promedios$Mean_B_Esc3, promedios$Theta_PDI_Esc3, promedios$RegDI_Esc3, promedios$RegDI_Correccion_Esc3
  ),
  SE = c(
    sd(resultados_simulacion$mean_A_esc1), sd(resultados_simulacion$mean_B_esc1), sd(resultados_simulacion$theta_PDI_esc1), SE_promedios$SE_RegDI_Esc1,
    sd(resultados_simulacion$mean_A_esc2), sd(resultados_simulacion$mean_B_esc2), sd(resultados_simulacion$theta_PDI_esc2), SE_promedios$SE_RegDI_Esc2, SE_promedios$SE_RegDI_Correccion_Esc2,
    sd(resultados_simulacion$mean_A_esc3), sd(resultados_simulacion$mean_B_esc3), sd(resultados_simulacion$theta_PDI_esc3), SE_promedios$SE_RegDI_Esc3, SE_promedios$SE_RegDI_Correccion_Esc3
  ),
  RMSE = c(
    sqrt((promedios$Mean_A_Esc1)^2 + var(resultados_simulacion$mean_A_esc1)),
    sqrt((promedios$Mean_B_Esc1)^2 + var(resultados_simulacion$mean_B_esc1)),
    sqrt((promedios$Theta_PDI_Esc1)^2 + var(resultados_simulacion$theta_PDI_esc1)),
    RMSEs$RMSE_RegDI_Esc1,
    sqrt((promedios$Mean_A_Esc2)^2 + var(resultados_simulacion$mean_A_esc2)),
    sqrt((promedios$Mean_B_Esc2)^2 + var(resultados_simulacion$mean_B_esc2)),
    sqrt((promedios$Theta_PDI_Esc2)^2 + var(resultados_simulacion$theta_PDI_esc2)),
    RMSEs$RMSE_RegDI_Esc2,
    RMSEs$RMSE_RegDI_Correccion_Esc2,
    sqrt((promedios$Mean_A_Esc3)^2 + var(resultados_simulacion$mean_A_esc3)),
    sqrt((promedios$Mean_B_Esc3)^2 + var(resultados_simulacion$mean_B_esc3)),
    sqrt((promedios$Theta_PDI_Esc3)^2 + var(resultados_simulacion$theta_PDI_esc3)),
    RMSEs$RMSE_RegDI_Esc3,
    RMSEs$RMSE_RegDI_Correccion_Esc3
  )
)

# ---- Redondear los valores a 2 decimales ----
resultados_finales <- resultados_finales %>%
  mutate(
    Sesgo = round(Sesgo, 2),
    SE = round(SE, 3),
    RMSE = round(RMSE, 3)
  )

# ---- Mostrar el data.frame final ----
print(resultados_finales)


   Escenario        Estimador Sesgo    SE  RMSE
1      Esc 1                A  0.00 0.033 0.033
2      Esc 1                B -0.11 0.012 0.112
3      Esc 1              PDI  0.00 0.025 0.025
4      Esc 1            RegDI  0.00 0.022 0.022
5      Esc 2                A  0.00 0.033 0.033
6      Esc 2                B -1.10 0.013 1.101
7      Esc 2              PDI -0.49 0.025 0.495
8      Esc 2            RegDI  0.00 0.025 0.025
9      Esc 2 RegDI_Correccion  0.00 0.025 0.025
10     Esc 3                A -1.00 0.034 0.999
11     Esc 3                B -0.11 0.012 0.112
12     Esc 3              PDI -0.50 0.025 0.505
13     Esc 3            RegDI -1.00 0.025 0.999
14     Esc 3 RegDI_Correccion  0.00 0.028 0.028


Simulación TOTAL

In [15]:
# Cargar librerías necesarias
library(survey)
library(dplyr)
library(knitr)
library(kableExtra)

# Parámetros de la simulación
n_sim <- 100       # Número de simulaciones
N <- 10000         # Tamaño de la población
size_a <- 1000     # Tamaño de la muestra probabilística A
size_muestra_B <- 5000 # Tamaño de la muestra no probabilística B

# Inicializar matrices para almacenar los resultados
estimates_mean_S_A <- numeric(n_sim)
estimates_mean_S_A_SE <- numeric(n_sim)
estimates_mean_S_B <- numeric(n_sim)
estimates_mean_S_B_SE <- numeric(n_sim)

estimates_RegDI <- numeric(n_sim)
estimates_RegDI_SE <- numeric(n_sim)
estimates_RegDI_X1 <- numeric(n_sim)
estimates_RegDI_X1_SE <- numeric(n_sim)
estimates_RegDI_e1 <- numeric(n_sim)
estimates_RegDI_e1_SE <- numeric(n_sim)

estimates_PC_X1 <- numeric(n_sim)
estimates_PC_X1_SE <- numeric(n_sim)
estimates_PC_e1 <- numeric(n_sim)
estimates_PC_e1_SE <- numeric(n_sim)

# Valor verdadero de la media poblacional
Y_true <- 3

# Iniciar la simulación
for (sim in 1:n_sim) {
  # Generar la población
  x_i <- rnorm(N, mean = 2, sd = 1)
  eta_i <- rnorm(N, mean = 0, sd = sqrt(0.51))
  y_i <- 3 + 0.7 * (x_i - 2) + eta_i
  
  # Generar variables auxiliares e_i y las variables indicadoras e1_i y e2_i
  rho <- 0.5
  v_i <- rnorm(N, mean = 0, sd = 1)
  e_i <- rho * x_i + sqrt(1 - rho^2) * v_i  # Variable correlacionada
  e1_i <- ifelse(e_i <= 1, 1, 0) # Variable indicadora e1i
  e2_i <- ifelse(e_i > 1, 1, 0) # Variable indicadora e2i
  
  # Generar las variables auxiliares x1_i y x2_i
  x1_i <- ifelse(x_i <= 2, 1, 0) 
  x2_i <- ifelse(x_i > 2, 1, 0)
  
  # Crear data.frame de la población
  poblacion <- data.frame(
    id = 1:N,
    x_i = x_i,
    y_i = y_i,
    e_i = e_i,
    e1_i = e1_i,
    e2_i = e2_i,
    x1_i = x1_i,
    x2_i = x2_i
  )
  
  # Seleccionar muestra probabilística A (S_A) de tamaño 1000
  set.seed(sim + 1000)  # Asegurar reproducibilidad
  indices_A <- sample(1:N, size = size_a, replace = FALSE)
  poblacion$muestra_A <- 0
  poblacion$muestra_A[indices_A] <- 1
  
  # Seleccionar muestra no probabilística B (S_B) de tamaño 5000
  # Estratificación basada en x_i
  stratum1 <- which(poblacion$x_i <= 2)
  stratum2 <- which(poblacion$x_i > 2)
  n_B1 <- 3000
  n_B2 <- 2000
  set.seed(sim + 2000)  # Asegurar reproducibilidad
  indices_B1 <- sample(stratum1, size = n_B1, replace = FALSE)
  indices_B2 <- sample(stratum2, size = n_B2, replace = FALSE)
  indices_B <- c(indices_B1, indices_B2)
  poblacion$muestra_B <- 0
  poblacion$muestra_B[indices_B] <- 1
  
  # Calcular los totales poblacionales conocidos para las variables auxiliares
  pop_totals_x <- c(
    x1_i = sum(poblacion$x1_i),
    x2_i = sum(poblacion$x2_i)
  )
  
  pop_totals_e <- c(
    e1_i = sum(poblacion$e1_i),
    e2_i = sum(poblacion$e2_i)
  )
  
  # Calcular Mean_S_A y Mean_S_B usando svymean, coef y SE
  design_A <- svydesign(ids = ~1, data = poblacion[poblacion$muestra_A == 1, ], weights = ~1)
  mean_S_A <- svymean(~y_i, design_A)
  estimates_mean_S_A[sim] <- coef(mean_S_A)
  estimates_mean_S_A_SE[sim] <- SE(mean_S_A)
  
  design_B <- svydesign(ids = ~1, data = poblacion[poblacion$muestra_B == 1, ], weights = ~1)
  mean_S_B <- svymean(~y_i, design_B)
  estimates_mean_S_B[sim] <- coef(mean_S_B)
  estimates_mean_S_B_SE[sim] <- SE(mean_S_B)
  
  # Calcular RegDI2 sin variables auxiliares
  result_RegDI <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = NULL,
    weights_A_col = NULL
  )
  estimates_RegDI[sim] <- result_RegDI$mean_RegDI
  estimates_RegDI_SE[sim] <- sqrt(result_RegDI$var_RegDI)
  
  # Calcular RegDI2 con (x1_i)
  result_RegDI_X1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("x1_i"),
    weights_A_col = NULL
  )
  estimates_RegDI_X1[sim] <- result_RegDI_X1$mean_RegDI
  estimates_RegDI_X1_SE[sim] <- sqrt(result_RegDI_X1$var_RegDI)
  
  # Calcular RegDI2 con (e1_i)
  result_RegDI_e1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("e1_i"),
    weights_A_col = NULL
  )
  estimates_RegDI_e1[sim] <- result_RegDI_e1$mean_RegDI
  estimates_RegDI_e1_SE[sim] <- sqrt(result_RegDI_e1$var_RegDI)
  
  # Calcular PC con variables auxiliares (x1_i, x2_i)
  PC_X1 <- PC_Estimator(
    data = poblacion,
    y_B_col = "y_i",
    sample_B_col = "muestra_B",
    aux_vars = c("x1_i","x2_i"),
    pop_totals = pop_totals_x,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL
  )
  estimates_PC_X1[sim] <- PC_X1$mean_PC
  estimates_PC_X1_SE[sim] <- PC_X1$SE_PC
  
  # Calcular PC con variables auxiliares (e1_i, e2_i)
  PC_e1 <- PC_Estimator(
    data = poblacion,
    y_B_col = "y_i",
    sample_B_col = "muestra_B",
    aux_vars = c("e1_i","e2_i"),
    pop_totals = pop_totals_e,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL
  )
  estimates_PC_e1[sim] <- PC_e1$mean_PC
  estimates_PC_e1_SE[sim] <- PC_e1$SE_PC
}

# Calcular las estadísticas resumen para cada estimador
calculate_stats <- function(estimates, true_value, se_estimates) {
  bias <- mean(estimates) - true_value
  avg_se <- mean(se_estimates)
  rmse <- sqrt(mean((estimates - true_value)^2))
  return(c(bias = bias, SE = avg_se, rmse = rmse))
}

# Crear una lista con todos los estimadores
estimators <- list(
  Mean_S_A = estimates_mean_S_A,
  Mean_S_B = estimates_mean_S_B,
  RegDI = estimates_RegDI,
  RegDI_X1 = estimates_RegDI_X1,
  RegDI_e1 = estimates_RegDI_e1,
  PC_X1 = estimates_PC_X1,
  PC_e1 = estimates_PC_e1
)

se_estimators <- list(
  Mean_S_A_SE = estimates_mean_S_A_SE,
  Mean_S_B_SE = estimates_mean_S_B_SE,
  RegDI_SE = estimates_RegDI_SE,
  RegDI_X1_SE = estimates_RegDI_X1_SE,
  RegDI_e1_SE = estimates_RegDI_e1_SE,
  PC_X1_SE = estimates_PC_X1_SE,
  PC_e1_SE = estimates_PC_e1_SE
)

# Calcular Bias, SE y RMSE para cada estimador
results_summary <- data.frame(
  Estimator = character(),
  Bias = numeric(),
  SE = numeric(),
  RMSE = numeric(),
  stringsAsFactors = FALSE
)

for (est_name in names(estimators)) {
  stats <- calculate_stats(estimators[[est_name]], Y_true, se_estimators[[paste0(est_name, "_SE")]])
  results_summary <- rbind(results_summary, data.frame(
    Estimator = est_name,
    Bias = round(stats["bias"], 4),
    SE = round(stats["SE"], 4),
    RMSE = round(stats["rmse"], 4),
    stringsAsFactors = FALSE
  ))
}

# Agregar la columna de Escenario (I)
results_summary <- results_summary %>%
  mutate(Scenario = "I") %>%
  select(Scenario, Estimator, Bias, SE, RMSE)

# Mostrar los resultados en una tabla
print(results_summary)



      Scenario Estimator    Bias     SE   RMSE
bias         I  Mean_S_A -0.0050 0.0315 0.0331
bias1        I  Mean_S_B -0.1118 0.0140 0.1124
bias2        I     RegDI -0.0014 0.0223 0.0235
bias3        I  RegDI_X1 -0.0014 0.0223 0.0235
bias4        I  RegDI_e1 -0.0014 0.0223 0.0235
bias5        I     PC_X1 -0.0003 0.0120 0.0134
bias6        I     PC_e1 -0.0935 0.0136 0.0942


Simulación con caso 2

In [16]:
PC_Estimator <- function(data,
                         y_A_col = NULL,      # Nombre de la variable objetivo en S_A
                         y_B_col = NULL,      # Nombre de la variable objetivo en S_B (puede ser NULL si no está observada)
                         sample_A_col,        # Indicador de pertenencia a S_A
                         sample_B_col,        # Indicador de pertenencia a S_B
                         aux_vars,            # Variables auxiliares para calibración
                         pop_totals,  # Totales poblacionales de variables auxiliares
                         N_total,
                         initial_weight = NULL, # Peso inicial para S_B
                         weights_B_col = NULL,  # Nombre de la columna de pesos iniciales para S_B
                         weights_A_col = NULL,  # Nombre de la columna de pesos para S_A
                         pred_vars = NULL,    # Variables para el modelo de predicción
                         model_type = "normal",# Tipo de modelo de predicción ("normal", "logistic", "custom")
                         scenario = 1         # Escenario: 1 o 2
                         ) {
  N <- N_total # Tamaño poblacional (aproximado)
  
  # Subconjunto de datos para S_B y S_A
  data_S_B <- data[data[[sample_B_col]] == 1, ]
  n_B <- nrow(data_S_B)
  
  data_S_A <- data[data[[sample_A_col]] == 1, ]
  n_A <- nrow(data_S_A)
  
  if (n_B == 0) stop("No hay unidades en la muestra S_B.")
  if (n_A == 0) stop("No hay unidades en la muestra S_A.")
  
  # Asignar pesos iniciales para S_B
  if (!is.null(weights_B_col)) {
    data_S_B$d_i_B <- data_S_B[[weights_B_col]]
  } else if (is.null(initial_weight)) {
    data_S_B$d_i_B <- N / n_B
  } else if (is.numeric(initial_weight)) {
    data_S_B$d_i_B <- initial_weight
  } else if (initial_weight == "uniform") {
    data_S_B$d_i_B <- 1
  } else {
    stop("initial_weight debe ser numérico o 'uniform'.")
  }
  
  # Asignar pesos para S_A
  if (!is.null(weights_A_col)) {
    data_S_A$d_i_A <- data_S_A[[weights_A_col]]
  } else {
    data_S_A$d_i_A <- N / n_A
  }
  
  # Crear fórmula de calibración
  calibration_formula <- as.formula(paste("~0 +", paste(aux_vars, collapse = " + ")))
  
  # Calibrar pesos para S_B
  design_S_B <- svydesign(ids = ~1, data = data_S_B, weights = ~d_i_B)
  calibrated_design_S_B <- calibrate(
    design = design_S_B,
    formula = calibration_formula,
    population = pop_totals,
    calfun = "linear"
  )
  weights_calibrated_B <- weights(calibrated_design_S_B)
  
  if (scenario == 1) {
    # Escenario 1: Variable objetivo observada en S_B
    mean_estimated <- svymean(as.formula(paste("~", y_B_col)), calibrated_design_S_B)
    mean_PC <- coef(mean_estimated)
    SE_PC <- SE(mean_estimated)
    
    return(list(
      estimator = mean_PC,
      SE = SE_PC,
      weights_B = weights_calibrated_B
    ))
    
  } else if (scenario == 2) {
    # Escenario 2: Variable objetivo no observada en S_B, necesitamos predecir y_i
    if (is.null(pred_vars)) stop("Debe proporcionar pred_vars para el modelo de predicción.")
    if (is.null(y_A_col)) stop("Debe proporcionar y_A_col para el modelo de predicción.")
    
    # Obtener datos en la intersección de S_A y S_B
    data_intersect <- data[data[[sample_A_col]] == 1 & data[[sample_B_col]] == 1, ]
    n_intersect <- nrow(data_intersect)
    
    if (n_intersect == 0) {
      stop("No hay unidades en la intersección de S_A y S_B para estimar el modelo.")
    }
    
    # Construir el modelo de predicción utilizando datos en la intersección
    if (model_type == "normal") {
      prediction_formula <- as.formula(paste(y_A_col, "~", paste(pred_vars, collapse = " + ")))
      model <- lm(prediction_formula, data = data_intersect)
    } else if (model_type == "logistic") {
      prediction_formula <- as.formula(paste(y_A_col, "~", paste(pred_vars, collapse = " + ")))
      model <- glm(prediction_formula, data = data_intersect, family = binomial)
    } else if (model_type == "custom") {
      stop("Modelo 'custom' no implementado actualmente.")
    } else {
      stop("model_type debe ser 'normal', 'logistic' o 'custom'.")
    }
    
    # Predecir y_i en S_B
    data_S_B$hat_y_i <- predict(model, newdata = data_S_B, type = ifelse(model_type == "logistic", "response", "response"))
    
    # Predecir y_i en S_A y calcular residuos
    data_S_A$hat_y_i <- predict(model, newdata = data_S_A, type = ifelse(model_type == "logistic", "response", "response"))
    data_S_A$residuals <- data_S_A[[y_A_col]] - data_S_A$hat_y_i
    
    # Calcular el estimador
    term1 <- sum(weights_calibrated_B * data_S_B$hat_y_i)
    term2 <- sum(data_S_A$d_i_A * data_S_A$residuals)
    estimator <- (term1 + term2) / N  # Dividir por N para obtener la media
    
    # Nota: La estimación del error estándar no se incluye en este código
    
    return(list(
      estimator = estimator,
      weights_B = weights_calibrated_B,
      model = model
    ))
    
  } else {
    stop("Escenario inválido. Debe ser 1 o 2.")
  }
}


In [17]:
# Cargar librerías necesarias
library(survey)
library(dplyr)
library(knitr)
library(kableExtra)

# Asegúrate de tener definidas las funciones RegDI2 y PC_Estimator según tus definiciones anteriores
# RegDI2 <- function(...) { ... }
# PC_Estimator <- function(...) { ... }

# Parámetros de la simulación
n_sim <- 1000             # Número de simulaciones
N <- 100000               # Tamaño de la población
size_a <- 1000           # Tamaño de la muestra probabilística A
size_muestra_B <- 50000   # Tamaño de la muestra no probabilística B

# Inicializar vectores para almacenar los resultados

# Escenario I
estimates_mean_S_A <- numeric(n_sim)
estimates_mean_S_A_SE <- numeric(n_sim)
estimates_mean_S_B <- numeric(n_sim)
estimates_mean_S_B_SE <- numeric(n_sim)

estimates_RegDI <- numeric(n_sim)
estimates_RegDI_SE <- numeric(n_sim)
estimates_RegDI_X1 <- numeric(n_sim)
estimates_RegDI_X1_SE <- numeric(n_sim)
estimates_RegDI_e1 <- numeric(n_sim)
estimates_RegDI_e1_SE <- numeric(n_sim)

estimates_PC_X1 <- numeric(n_sim)
estimates_PC_X1_SE <- numeric(n_sim)
estimates_PC_e1 <- numeric(n_sim)
estimates_PC_e1_SE <- numeric(n_sim)

# Escenario II
estimates_RegDI_corr1_X1 <- numeric(n_sim)
estimates_RegDI_corr1_X1_SE <- numeric(n_sim)
estimates_RegDI_corr1_e1 <- numeric(n_sim)
estimates_RegDI_corr1_e1_SE <- numeric(n_sim)

estimates_PC2_X1 <- numeric(n_sim)
estimates_PC2_e1 <- numeric(n_sim)

# Valor verdadero de la media poblacional
Y_true <- 3

# Iniciar la simulación
for (sim in 1:n_sim) {
  # Generar la población
  set.seed(sim)  # Asegurar reproducibilidad por iteración
  x_i <- rnorm(N, mean = 2, sd = 1)
  eta_i <- rnorm(N, mean = 0, sd = sqrt(0.51))
  y_i <- 3 + 0.7 * (x_i - 2) + eta_i
  
  # Generar la versión contaminada de y_i (tilde_y_i)
  tilde_y_i <- 2 + 0.9 * (y_i - 3) + rnorm(N, mean = 0, sd = 0.5)
  
  # Generar variables auxiliares e_i y las variables indicadoras e1_i y e2_i
  rho <- 0.5
  v_i <- rnorm(N, mean = 0, sd = 1)
  e_i <- rho * x_i + sqrt(1 - rho^2) * v_i  # Variable correlacionada
  e1_i <- ifelse(e_i <= 1, 1, 0)            # Variable indicadora e1_i
  e2_i <- ifelse(e_i > 1, 1, 0)             # Variable indicadora e2_i
  
  # Generar las variables auxiliares x1_i y x2_i
  x1_i <- ifelse(x_i <= 2, 1, 0) 
  x2_i <- ifelse(x_i > 2, 1, 0)
  
  # Crear data.frame de la población
  poblacion <- data.frame(
    id = 1:N,
    x_i = x_i,
    y_i = y_i,
    tilde_y_i = tilde_y_i,
    e_i = e_i,
    e1_i = e1_i,
    e2_i = e2_i,
    x1_i = x1_i,
    x2_i = x2_i
  )
  
  # Seleccionar muestra probabilística A (S_A) de tamaño 1000
  set.seed(sim + 1000)  # Asegurar reproducibilidad
  indices_A <- sample(1:N, size = size_a, replace = FALSE)
  poblacion$muestra_A <- 0
  poblacion$muestra_A[indices_A] <- 1
  
  # Seleccionar muestra no probabilística B (S_B) de tamaño 5000
  # Estratificación basada en x_i
  stratum1 <- which(poblacion$x_i <= 2)
  stratum2 <- which(poblacion$x_i > 2)
  n_B1 <- 3000
  n_B2 <- 2000
  set.seed(sim + 2000)  # Asegurar reproducibilidad
  indices_B1 <- sample(stratum1, size = n_B1, replace = FALSE)
  indices_B2 <- sample(stratum2, size = n_B2, replace = FALSE)
  indices_B <- c(indices_B1, indices_B2)
  poblacion$muestra_B <- 0
  poblacion$muestra_B[indices_B] <- 1
  
  # Calcular los totales poblacionales conocidos para las variables auxiliares
  pop_totals_x <- c(
    x1_i = sum(poblacion$x1_i),
    x2_i = sum(poblacion$x2_i)
  )
  
  pop_totals_e <- c(
    e1_i = sum(poblacion$e1_i),
    e2_i = sum(poblacion$e2_i)
  )
  
  # **Estimación de Medias Simples para S_A y S_B**
  design_A <- svydesign(ids = ~1, data = poblacion[poblacion$muestra_A == 1, ], weights = ~1)
  mean_S_A <- svymean(~y_i, design_A)
  estimates_mean_S_A[sim] <- coef(mean_S_A)
  estimates_mean_S_A_SE[sim] <- SE(mean_S_A)
  
  design_B <- svydesign(ids = ~1, data = poblacion[poblacion$muestra_B == 1, ], weights = ~1)
  mean_S_B <- svymean(~y_i, design_B)
  estimates_mean_S_B[sim] <- coef(mean_S_B)
  estimates_mean_S_B_SE[sim] <- SE(mean_S_B)
  
  # **Estimación con RegDI2 sin variables auxiliares (Escenario I)**
  result_RegDI <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = NULL,
    weights_A_col = NULL
  )
  estimates_RegDI[sim] <- result_RegDI$mean_RegDI
  estimates_RegDI_SE[sim] <- sqrt(result_RegDI$var_RegDI)
  
  # **Estimación con RegDI2 con x1_i (Escenario I)**
  result_RegDI_X1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("x1_i"),
    weights_A_col = NULL
  )
  estimates_RegDI_X1[sim] <- result_RegDI_X1$mean_RegDI
  estimates_RegDI_X1_SE[sim] <- sqrt(result_RegDI_X1$var_RegDI)
  
  # **Estimación con RegDI2 con e1_i (Escenario I)**
  result_RegDI_e1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 0,
    aux_vars = c("e1_i"),
    weights_A_col = NULL
  )
  estimates_RegDI_e1[sim] <- result_RegDI_e1$mean_RegDI
  estimates_RegDI_e1_SE[sim] <- sqrt(result_RegDI_e1$var_RegDI)
  
  # **Estimación con PC_Estimator (Escenario I) con (x1_i, x2_i)**
  result_PC_X1 <- PC_Estimator(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    sample_A_col = "muestra_A",
    sample_B_col = "muestra_B",
    aux_vars = c("x1_i", "x2_i"),
    pop_totals = pop_totals_x,
    N_total = N,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL,
    pred_vars = NULL,
    model_type = "normal",
    scenario = 1
  )
  estimates_PC_X1[sim] <- result_PC_X1$estimator
  estimates_PC_X1_SE[sim] <- result_PC_X1$SE
  
  # **Estimación con PC_Estimator (Escenario I) con (e1_i, e2_i)**
  result_PC_e1 <- PC_Estimator(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    sample_A_col = "muestra_A",
    sample_B_col = "muestra_B",
    aux_vars = c("e1_i", "e2_i"),
    pop_totals = pop_totals_e,
    N_total = N,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL,
    pred_vars = NULL,
    model_type = "normal",
    scenario = 1
  )
  estimates_PC_e1[sim] <- result_PC_e1$estimator
  estimates_PC_e1_SE[sim] <- result_PC_e1$SE
  
  # **Estimación con RegDI2 con apply_correction = 1 y x1_i (Escenario II)**
  result_RegDI_corr1_X1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 1,            # Aplicar corrección de errores en la muestra B
    aux_vars = c("x1_i"),            # Incluir x1_i como variable auxiliar
    weights_A_col = NULL
  )
  estimates_RegDI_corr1_X1[sim] <- result_RegDI_corr1_X1$mean_RegDI
  estimates_RegDI_corr1_X1_SE[sim] <- sqrt(result_RegDI_corr1_X1$var_RegDI)
  
  # **Estimación con RegDI2 con apply_correction = 1 y e1_i (Escenario II)**
  result_RegDI_corr1_e1 <- RegDI2(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",
    size_a = size_a,
    size_muestra_B = size_muestra_B,
    N = N,
    apply_correction = 1,            # Aplicar corrección de errores en la muestra B
    aux_vars = c("e1_i"),            # Incluir e1_i como variable auxiliar
    weights_A_col = NULL
  )
  estimates_RegDI_corr1_e1[sim] <- result_RegDI_corr1_e1$mean_RegDI
  estimates_RegDI_corr1_e1_SE[sim] <- sqrt(result_RegDI_corr1_e1$var_RegDI)
  
  # **Estimación con PC_Estimator (Escenario II) con (x1_i, x2_i)**
  result_PC2_X1 <- PC_Estimator(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",                # y_B_col se usa internamente para la calibración
    sample_A_col = "muestra_A",
    sample_B_col = "muestra_B",
    aux_vars = c("x1_i", "x2_i"),
    pop_totals = pop_totals_x,
    N_total = N,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL,
    pred_vars = c("tilde_y_i"),      # Variable contaminada
    model_type = "normal",
    scenario = 2
  )
  estimates_PC2_X1[sim] <- result_PC2_X1$estimator
  # **SE y RMSE no se calculan en Escenario II para PC**
  
  # **Estimación con PC_Estimator (Escenario II) con (e1_i, e2_i)**
  result_PC2_e1 <- PC_Estimator(
    data = poblacion,
    y_A_col = "y_i",
    y_B_col = "y_i",                # y_B_col se usa internamente para la calibración
    sample_A_col = "muestra_A",
    sample_B_col = "muestra_B",
    aux_vars = c("e1_i", "e2_i"),
    pop_totals = pop_totals_e,
    N_total = N,
    initial_weight = N / size_muestra_B,
    weights_B_col = NULL,
    pred_vars = c("tilde_y_i"),      # Variable contaminada
    model_type = "normal",
    scenario = 2
  )
  estimates_PC2_e1[sim] <- result_PC2_e1$estimator
  # **SE y RMSE no se calculan en Escenario II para PC**
}

# Calcular las estadísticas resumen para cada estimador
calculate_stats <- function(estimates, true_value, se_estimates) {
  bias <- mean(estimates) - true_value
  avg_se <- mean(se_estimates, na.rm = TRUE)
  rmse <- sqrt(mean((estimates - true_value)^2))
  return(c(bias = bias, SE = avg_se, rmse = rmse))
}

# Crear una lista con todos los estimadores
estimators <- list(
  # Escenario I
  Mean_S_A = estimates_mean_S_A,
  Mean_S_B = estimates_mean_S_B,
  RegDI = estimates_RegDI,
  RegDI_X1 = estimates_RegDI_X1,
  RegDI_e1 = estimates_RegDI_e1,
  PC_X1 = estimates_PC_X1,
  PC_e1 = estimates_PC_e1,
  
  # Escenario II
  RegDI_corr1_X1 = estimates_RegDI_corr1_X1,
  RegDI_corr1_e1 = estimates_RegDI_corr1_e1,
  PC2_X1 = estimates_PC2_X1,
  PC2_e1 = estimates_PC2_e1
)

# Crear una lista con los SE de los estimadores
se_estimators <- list(
  # Escenario I
  Mean_S_A_SE = estimates_mean_S_A_SE,
  Mean_S_B_SE = estimates_mean_S_B_SE,
  RegDI_SE = estimates_RegDI_SE,
  RegDI_X1_SE = estimates_RegDI_X1_SE,
  RegDI_e1_SE = estimates_RegDI_e1_SE,
  PC_X1_SE = estimates_PC_X1_SE,
  PC_e1_SE = estimates_PC_e1_SE,
  
  # Escenario II
  RegDI_corr1_X1_SE = estimates_RegDI_corr1_X1_SE,
  RegDI_corr1_e1_SE = estimates_RegDI_corr1_e1_SE,
  PC2_X1_SE = rep(NaN, n_sim),   # SE no disponible en Escenario II para PC
  PC2_e1_SE = rep(NaN, n_sim)    # SE no disponible en Escenario II para PC
)

# Calcular Bias, SE y RMSE para cada estimador
results_summary <- data.frame(
  Scenario = character(),
  Estimator = character(),
  Bias = numeric(),
  SE = numeric(),
  RMSE = numeric(),
  stringsAsFactors = FALSE
)

for (est_name in names(estimators)) {
  # Determinar el escenario basado en el nombre del estimador
  if (grepl("^PC2_", est_name)) {
    scenario <- "II"
    est_simple_name <- sub("^PC2_", "PC (Escenario II) ", est_name)
  } else if (grepl("^RegDI_corr1_", est_name)) {
    scenario <- "II"
    est_simple_name <- sub("^RegDI_corr1_", "RegDI Corr1 (Escenario II) ", est_name)
  } else {
    scenario <- "I"
    est_simple_name <- est_name
  }
  
  # Obtener los SE correspondientes
  se_est <- se_estimators[[paste0(est_name, "_SE")]]
  
  # Calcular estadísticas
  stats <- calculate_stats(
    estimates = estimators[[est_name]],
    true_value = Y_true,
    se_estimates = se_est
  )
  
  # Para PC en Escenario II, establecer SE y RMSE como NA
  if (grepl("^PC2_", est_name)) {
    stats["SE"] <- NA
    stats["rmse"] <- NA
  }
  
  # Agregar al resumen
  results_summary <- rbind(results_summary, data.frame(
    Scenario = scenario,
    Estimator = est_simple_name,
    Bias = round(stats["bias"], 4),
    SE = ifelse(is.na(stats["SE"]), NA, round(stats["SE"], 4)),
    RMSE = ifelse(is.na(stats["rmse"]), NA, round(stats["rmse"], 4)),
    stringsAsFactors = FALSE
  ))
}

# Ordenar la tabla para una mejor visualización
results_summary <- results_summary %>%
  arrange(Scenario, Estimator)



In [18]:
results_summary

,Scenario,Estimator,Bias,SE,RMSE
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
bias,I,Mean_S_A,0.0000,0.0316,0.0314
bias1,I,Mean_S_B,-0.1121,0.0141,0.1127
bias5,I,PC_X1,-0.0004,0.0120,0.0119
bias6,I,PC_e1,-0.0939,0.0136,0.0946
bias2,I,RegDI,0.0001,0.0308,0.0305
bias3,I,RegDI_X1,0.0001,0.0308,0.0305
bias4,I,RegDI_e1,0.0001,0.0308,0.0305
bias9,II,PC (Escenario II) X1,0.0002,NA,NA
bias10,II,PC (Escenario II) e1,-0.0708,NA,NA
